##Установка и импортирование библиотек##



In [1]:
!pip install catboost

     |████████████████████████████████| 65.7MB 82kB/s 


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score

import catboost as cb
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import GridSearchCV

from itertools import product, chain

#Требуется добавить файл paramsearch.py в проект
#from paramsearch import paramsearch

## Загрузка данных ##



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/Хакатон/dataset2015_2019.csv')

In [ ]:
data.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point
0,0,Point (907753.82834152 6554634.46488151),907753.828342,6.554634e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (907748.69100314 6554212.06490037),Point (907410.62452802 6554909.79074054),Point (908089.34400838 6554953.05455597),Point (907563.65755459 6554039.35762193)
1,1,Point (906932.34642625 6550537.14396784),906932.346426,6.550537e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (906980.7207283 6550304.97315954),Point (906736.32315476 6550335.52813463),Point (906595.72670333 6550598.70621165),Point (907032.95563938 6550121.6369531)


## Эвристики для предсказаний данных без модели ##

### Эвристика по закономерностям последовательностей культур на основе анализа *данных* ###

In [5]:
#Функция для предсказания культуры в следующем году на основании культур предыдущих 4 лет
def predict_next_culture_4_years_list(l0, l1, l2, l3):
  if l0 == l1 != l2 and l2 == l3:
     return 'Unknown'
  if l0 == l1 == l2 == l3:
     return l0
  if l0 == l2 and l1 == l3:
     return l0
  if l0 == l1 != l2 and l2 == l3:
     return l0
  return 'Unknown'

In [6]:
print(predict_next_culture_4_years_list(data.iloc[0]['CODE_CULTU_2015'],data.iloc[0]['CODE_CULTU_2016'],data.iloc[0]['CODE_CULTU_2017'],data.iloc[0]['CODE_CULTU_2018']))

PPH


### Эвристика по уникальным последовательностям ###

In [7]:
#Функция для получения списка уникальных последовательностей (после 4 культур предыдущих лет всегда идет одна и та же 5-ая)
def get_unique_chains(data):
  g = data.groupby(['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])['CODE_CULTU_2019'].count()
  g = g.add_suffix('').reset_index()
  g_uniq = g[g['CODE_CULTU_2019'] == 1]
  g_pred = pd.merge(g_uniq, data,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])
  g_pred = g_pred.reindex(columns=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018','CODE_CULTU_2019_y'])
  g_pred.rename(columns = {'CODE_CULTU_2019_y': 'CODE_CULTU_2019_predict'}, inplace = True)
  return g_pred

In [8]:
#Функция для получения списка уникальных последовательностей для тестовой выборки
def get_unique_chains_test(data):
  g = data.groupby(['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018']).count()
  g = g.add_suffix('').reset_index()
  return g

In [9]:
print(get_unique_chains(data))

      CODE_CULTU_2015 CODE_CULTU_2016  ... CODE_CULTU_2018 CODE_CULTU_2019_predict
0                 ACA             FLA  ...             FLA                     FLA
1                 ACA             FLP  ...             FLP                     FLP
2                 ACA             MIS  ...             SNE                     SNE
3                 ACA             SNE  ...             SNE                     SNE
4                 AGR             BOR  ...             SNE                     SNE
...               ...             ...  ...             ...                     ...
82546             VRT             VRT  ...             SOG                     BTH
82547             XFE             PTR  ...             BTH                     PTR
82548             XFE             PTR  ...             PPH                     PRL
82549             XFE             XFE  ...             PRL                     CZH
82550             XFE             XFE  ...             XFE                     MIE

[82

### Предсказания по эвристикам на обучающей выборке ###

In [10]:
g_pred = get_unique_chains(data)

In [11]:
#Получаем предсказания по эвристике по уникальным последовательностям
data_pred = pd.merge(data, g_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])

In [12]:
print(data_pred['CODE_CULTU_2019_predict'].value_counts().sum())
print(data_pred['CODE_CULTU_2019_predict'].value_counts())

82551
BTH    11247
PTR     8134
LUZ     4978
MLG     4003
MIS     3902
       ...  
PIS        1
MRG        1
BUR        1
PMV        1
BAR        1
Name: CODE_CULTU_2019_predict, Length: 218, dtype: int64


In [13]:
#Сохраняем первую часть, предсказанную по эвристике по уникальным последовательностям
data_part1 = data_pred[data_pred.CODE_CULTU_2019_predict.notnull()].copy()

In [14]:
data_part1.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point,CODE_CULTU_2019_predict
50,50,Point (859900.11098517 6591026.78278985),859900.110985,6.591027e+06,3,1,BTH,MID,MIS,MIS,BTH,1,2,2,2,1,abcca,abbba,8b,Cfa,ff13,Point (859812.49760217 6591185.50103266),Point (859719.0802948 6590975.56193138),Point (860081.26015362 6591098.77150968),Point (860019.18559373 6590736.93947271),BTH
74,74,Point (848022.43324874 6589869.38021483),848022.433249,6.589869e+06,4,1,AVP,SGH,MIS,MIS,ORP,4,4,2,2,3,abccd,aabbc,8b,Cfa,ff13,Point (848060.93345233 6589936.96049742),Point (847937.44700784 6589858.58590304),Point (848069.49079389 6590019.03797348),Point (848120.55312173 6589719.68664306),ORP


In [15]:
#Формируем вторую часть для предсказаний по эверистике по закономерностям последовательностей культур на основе анализа
data_part2 = data_pred[data_pred.CODE_CULTU_2019_predict.isna()].copy()

In [17]:
data_part2.head(2)

,Unnamed: 0,centroid,x,y,RotationLength,RotationCount,CODE_CULTU_2015,CODE_CULTU_2016,CODE_CULTU_2017,CODE_CULTU_2018,CODE_CULTU_2019,CODE_GROUP_2015,CODE_GROUP_2016,CODE_GROUP_2017,CODE_GROUP_2018,CODE_GROUP_2019,cultuCode,groupCode,hzs,kg_id,ff_id,1 Nearest Neigbour Point,2 Nearest Neigbour Point,3 Nearest Neigbour Point,4 Nearest Neigbour Point,CODE_CULTU_2019_predict
0,0,Point (907753.82834152 6554634.46488151),907753.828342,6.554634e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (907748.69100314 6554212.06490037),Point (907410.62452802 6554909.79074054),Point (908089.34400838 6554953.05455597),Point (907563.65755459 6554039.35762193),NaN
1,1,Point (906932.34642625 6550537.14396784),906932.346426,6.550537e+06,1,5,PPH,PPH,PPH,PPH,PPH,18,18,18,18,18,aaaaa,aaaaa,7b,Dfb,ff12,Point (906980.7207283 6550304.97315954),Point (906736.32315476 6550335.52813463),Point (906595.72670333 6550598.70621165),Point (907032.95563938 6550121.6369531),NaN


In [18]:
#Получаем предсказания по эвристике по закономерностям последовательностей культур на основе анализа данных
data_part2['CODE_CULTU_2019_predict'] = data_part2.apply(lambda x: predict_next_culture_4_years_list(x['CODE_CULTU_2015'],x['CODE_CULTU_2016'],x['CODE_CULTU_2017'],x['CODE_CULTU_2018']), axis=1, result_type='expand')

In [19]:
print(data_part2[data_part2['CODE_CULTU_2019_predict'] != 'Unknown']['CODE_CULTU_2019_predict'].value_counts().sum())
print(data_part2[data_part2['CODE_CULTU_2019_predict'] != 'Unknown']['CODE_CULTU_2019_predict'].value_counts())

2078679
PPH    1067851
VRC     181807
J6S     100032
SNE      85849
PRL      79558
        ...   
LIP          2
BAS          2
NVF          2
CZP          2
PAN          2
Name: CODE_CULTU_2019_predict, Length: 159, dtype: int64


In [ ]:
#Формирование выборки для оценки точности
data_eval = data_part2[data_part2['CODE_CULTU_2019_predict'] != 'Unknown']

In [ ]:
print(accuracy_score(data_eval['CODE_CULTU_2019'],data_eval['CODE_CULTU_2019_predict']))

0.9479198086861896


In [21]:
# Формирование набора данных, по которому не удалось предсказать значения с помощью эвристик ###
data_part3 = data_part2[data_part2['CODE_CULTU_2019_predict'] == 'Unknown']

In [22]:
#Формирование набора данных с предсказанными по эвристикам значениями
data_part2 = data_part2[data_part2['CODE_CULTU_2019_predict'] != 'Unknown']

## Моделирование ##

###Full CatBoost###

In [23]:
X_full = data_part3[[
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018']]

X_full = X_full.replace(np.nan, 'unknown', regex=True)

Y_full = data_part3['CODE_CULTU_2019']

In [25]:
cat_features_full = [0, 1, 2, 3, 4, 5, 6, 7]

In [26]:
train_dataset_full = Pool(data=X_full,
                     label=Y_full,
                     cat_features=cat_features_full)

eval_dataset_full = Pool(data=X_full,
                    label=Y_full,
                    cat_features=cat_features_full)

model_full = CatBoostClassifier(iterations=200,
                           #learning_rate=0.25,
                           depth=6,
                           loss_function='MultiClass',  task_type='GPU')

#Обучение модели
model_full.fit(train_dataset_full)  
#Получение предсказаний
preds_class_full = model_full.predict(eval_dataset_full)

Learning rate set to 0.5
0:	learn: 14.3086393	total: 8.74s	remaining: 28m 58s
1:	learn: 1144.3186298	total: 18.7s	remaining: 30m 50s
2:	learn: 3486.0962125	total: 27.6s	remaining: 30m 15s
3:	learn: 3452.6349205	total: 36.5s	remaining: 29m 48s
4:	learn: 3960.3020747	total: 45.3s	remaining: 29m 27s
5:	learn: 3204.9036430	total: 54.2s	remaining: 29m 11s
6:	learn: 2687.8177183	total: 1m 3s	remaining: 28m 57s
7:	learn: 2608.1470362	total: 1m 11s	remaining: 28m 44s
8:	learn: 2573.3483070	total: 1m 20s	remaining: 28m 32s
9:	learn: 2405.0925732	total: 1m 29s	remaining: 28m 21s
10:	learn: 2389.2164499	total: 1m 38s	remaining: 28m 10s
11:	learn: 2245.7084125	total: 1m 47s	remaining: 28m
12:	learn: 2293.1847928	total: 1m 56s	remaining: 27m 50s
13:	learn: 2179.5343285	total: 2m 4s	remaining: 27m 40s
14:	learn: 2102.4390145	total: 2m 13s	remaining: 27m 30s
15:	learn: 1983.8269644	total: 2m 22s	remaining: 27m 20s
16:	learn: 2024.6885429	total: 2m 31s	remaining: 27m 11s
17:	learn: 2066.7026350	total:

In [ ]:
preds_class_full = model_full.predict(eval_dataset_full)

In [ ]:
print(accuracy_score(Y_full,preds_class_full))

0.572658839522446


## Предсказания по тестовой выборке ##

In [27]:
test2020 = pd.read_csv('/content/drive/MyDrive/Хакатон/testWithCodes2015-2019.csv')

In [28]:
#Смещаем даты на год, т.к. предсказываем на 2020 по данным 2019-2016, для упрощения пайплана считаем, что предсказываем на 2019 по данным 2018-2015
test2020.rename(columns = {'CODE_CULTU_2015': 'CODE_CULTU_2014', 'CODE_GROUP_2015': 'CODE_GROUP_2014'}, inplace = True)
test2020.rename(columns = {'CODE_CULTU_2016': 'CODE_CULTU_2015', 'CODE_GROUP_2016': 'CODE_GROUP_2015'}, inplace = True)
test2020.rename(columns = {'CODE_CULTU_2017': 'CODE_CULTU_2016', 'CODE_GROUP_2017': 'CODE_GROUP_2016'}, inplace = True)
test2020.rename(columns = {'CODE_CULTU_2018': 'CODE_CULTU_2017', 'CODE_GROUP_2018': 'CODE_GROUP_2017'}, inplace = True)
test2020.rename(columns = {'CODE_CULTU_2019': 'CODE_CULTU_2018', 'CODE_GROUP_2019': 'CODE_GROUP_2018'}, inplace = True)

In [29]:
g_test_pred = get_unique_chains_test(test2020)

In [30]:
#Получает предсказания для тестовой выборки из уникальных последовательностей обучающей выборки
g_test_pred = pd.merge(g_test_pred, g_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])


In [31]:
g_test_pred = g_test_pred.reindex(columns=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018','CODE_CULTU_2019_predict'])

In [32]:
print(g_test_pred['CODE_CULTU_2019_predict'].value_counts().sum())
print(g_test_pred['CODE_CULTU_2019_predict'].value_counts())

1
GFP    1
Name: CODE_CULTU_2019_predict, dtype: int64


In [33]:
#Получаем предсказания по эвристике по уникальным последовательностям для тестовой выборки
test_pred = pd.merge(test2020, g_test_pred,  how='left', left_on=['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'], right_on = ['CODE_CULTU_2015','CODE_CULTU_2016','CODE_CULTU_2017','CODE_CULTU_2018'])

In [34]:
print(test_pred['CODE_CULTU_2019_predict'].value_counts().sum())
print(test_pred['CODE_CULTU_2019_predict'].value_counts())

1
GFP    1
Name: CODE_CULTU_2019_predict, dtype: int64


In [35]:
#Сохраняем первую часть, предсказанную по эвристике по уникальным последовательностям
test_part1 = test_pred[test_pred.CODE_CULTU_2019_predict.notnull()].copy()

In [36]:
test_part1.head(2)

,Unnamed: 0,CODE_CULTU_2018,CODE_GROUP_2018,centroid,CODE_CULTU_2017,CODE_GROUP_2017,CODE_CULTU_2016,CODE_GROUP_2016,CODE_CULTU_2015,CODE_GROUP_2015,CODE_CULTU_2014,CODE_GROUP_2014,cultuCode,groupCode,CODE_CULTU_2019_predict
11,11,GFP,16,Point (665214.065978 7101594.32914311),GFP,16,FAG,16,FAG,16,FAG,16,aabbb,aaaaa,GFP


In [37]:
#Формируем вторую часть для предсказаний по эверистике по закономерностям последовательностей культур на основе анализа
test_part2 = test_pred[test_pred.CODE_CULTU_2019_predict.isna()].copy()

In [38]:
#Получаем предсказания по эвристике по закономерностям последовательностей культур на основе анализа данных
test_part2['CODE_CULTU_2019_predict'] = test_part2.apply(lambda x: predict_next_culture_4_years_list(x['CODE_CULTU_2015'],x['CODE_CULTU_2016'],x['CODE_CULTU_2017'],x['CODE_CULTU_2018']), axis=1, result_type='expand')

In [39]:
print(test_part2['CODE_CULTU_2019_predict'].value_counts().sum())
print(test_part2['CODE_CULTU_2019_predict'].value_counts())

26
Unknown    13
BTA         9
J6S         1
FAG         1
GFP         1
PPH         1
Name: CODE_CULTU_2019_predict, dtype: int64


In [40]:
#Формирование набора данных, по которому не удалось предсказать значения с помощью эвристик ###
test_part3 = test_part2[test_part2['CODE_CULTU_2019_predict'] == 'Unknown']
test_part3 = test_part3.add_suffix('').reset_index()

In [41]:
#Формирование набора данных с предсказанными по эвристикам значениями
test_part2 = test_part2[test_part2['CODE_CULTU_2019_predict'] != 'Unknown']

In [42]:
#Признаки для модели для тестовой выборки
X_test = test_part3[[
       'CODE_CULTU_2015', 'CODE_CULTU_2016', 'CODE_CULTU_2017',
       'CODE_CULTU_2018', 'CODE_GROUP_2015',
       'CODE_GROUP_2016', 'CODE_GROUP_2017', 'CODE_GROUP_2018']]

X_test = X_test.replace(np.nan, 'unknown', regex=True)

cat_features_test = [0, 1, 2, 3, 4, 5, 6, 7]

In [43]:
#Получение предсказаний
test_dataset = Pool(data=X_test,
                    cat_features=cat_features_test)

preds_class_test = model_full.predict(test_dataset)

In [44]:
test_part3['CODE_CULTU_2019_predict'] = preds_class_test

Корректировка явных ошибок CatBoost (Предсказание значения, которое не встречалось в выборке, заменяем на последнее значение в выборке)

In [45]:
test_part3_1 = test_part3.loc[(test_part3['CODE_CULTU_2019_predict'] != test_part3['CODE_CULTU_2015']) & (test_part3['CODE_CULTU_2019_predict'] != test_part3['CODE_CULTU_2016']) & (test_part3['CODE_CULTU_2019_predict'] != test_part3['CODE_CULTU_2017']) & (test_part3['CODE_CULTU_2019_predict'] != test_part3['CODE_CULTU_2018']) ]

In [46]:
test_part3_2 = test_part3.loc[(test_part3['CODE_CULTU_2019_predict'] == test_part3['CODE_CULTU_2015']) | (test_part3['CODE_CULTU_2019_predict'] == test_part3['CODE_CULTU_2016']) | (test_part3['CODE_CULTU_2019_predict'] == test_part3['CODE_CULTU_2017']) | (test_part3['CODE_CULTU_2019_predict'] == test_part3['CODE_CULTU_2018']) ]

In [47]:
test_part3_1['CODE_CULTU_2019_predict'] = test_part3_1['CODE_CULTU_2018']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
#Объединение данных
final_data_test = pd.concat([test_part1,test_part2,test_part3])
final_data_test = final_data_test[['centroid','CODE_CULTU_2019_predict']]
final_data_test.rename(columns = {'CODE_CULTU_2019_predict': 'CODE_CULTU_2020'}, inplace = True)

In [ ]:
#Формирование файла ответов
pd.DataFrame(final_data_test[['centroid','CODE_CULTU_2020']]).to_csv("/content/drive/MyDrive/Хакатон/predict2020.csv",index=False,header=True)